# Vamos a leer el cluster 4 y generar el promedio

vamos por paso

En este caso no voy a generar las covariables que generó Daiana

Voy a probar que corra, si corre luego le agrego las variables y veo los cambios en Kaggle




## Preparo todo

In [1]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend



#import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

In [3]:
!pip install autogluon.timeseries

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import nbformat
from autogluon.timeseries import TimeSeriesPredictor
from autogluon.timeseries import TimeSeriesDataFrame
import autogluon
from autogluon.core.metrics import make_scorer

## Levanto el dataset
vamos a hacer el promedio de los últimos 12 meses por producto

In [13]:
# me traigo el archivo ya procesado con la duración completa
df_cluster_3 = pd.read_parquet(DATOS_DIR +'df_cluster_3.parquet')
df_cluster_3.head()
# productos a validar Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# me traigo el clasificador de clusters


In [14]:
df_cluster_3.shape

(174785, 45)

In [15]:
# me quedo con las filas de df_cluster_3 cuyo product_id esté en productos_validar
df_cluster_3 = df_cluster_3[df_cluster_3['product_id'].isin(productos_validar['product_id'])]

In [16]:
df_cluster_3.shape

(150526, 45)

In [17]:
df_cluster_3.dtypes


,0
customer_id,int64
product_id,int64
date_mes,period[M]
first_customer_date,period[M]
last_customer_date,period[M]
first_product_date,period[M]
last_product_date,period[M]
tn,float64
dif_tn,float64
dif_por,float64


## Ajustes propios de autogluon

In [20]:
# ===================================================================
# 3. DEFINICIÓN DE ROLES DE LAS COLUMNAS
# ===================================================================
# Define qué columnas usarás para cada cosa. Esta es la parte clave.

# Columna que identifica de forma única cada serie temporal
item_id_col = "product_cust_id"

# Columna con la fecha/hora
timestamp_col = "date_mes"

# La variable que queremos predecir
target_col = "tn"

# COVARIABLES CONOCIDAS: Features cuyos valores PUEDES CONOCER en el futuro.
# Ejemplos: Días laborables, feriados, campañas de marketing planificadas, proyecciones de inflación, precio del dólar futuro.
known_covariate_cols = [
    "dias_laborables_arg",
    "anio",
    "mes",
    'secuencia',
    'trimestre'# Ejemplo: 1 si hay un evento especial planeado, 0 si no
    # Añade aquí otras como 'Infl', 'Expect', 'dias_laborables_arg' si tienes proyecciones
]

# CARACTERÍSTICAS ESTÁTICAS: Features que describen la serie pero no cambian en el tiempo.
# Ejemplos: La categoría de un producto, el ID de un cliente, la marca.
# AutoGluon las detectará automáticamente si no varían, pero es bueno ser explícito.
static_features_cols = [
    "cat1",
    "cat1_cat2",
    "cat1_cat2_cat3"
    # etc.
]
#### vamos a confiar en la magia

# COVARIABLES PASADAS (futuro desconocido)
# ¡No necesitas una lista para estas! Simplemente estarán en el DataFrame.
# AutoGluon las usará automáticamente si no están en 'known_covariate_cols'.
# Ejemplos de tu lista: VtaGral, VtaGralCte, Bebidas, Almacén, stock_final, etc.


In [22]:
# ===================================================================
# 4. CONVERSIÓN A TIMESERIESDATAFRAME
# ===================================================================
# AutoGluon necesita que los datos estén en su formato específico.
# Las características estáticas se pasan por separado al crear el DataFrame.
df_cluster_3['date_mes'] = df_cluster_3['date_mes'].dt.to_timestamp()
all_cols = [item_id_col, timestamp_col, target_col] + known_covariate_cols
static_feature_cols_present = [col for col in static_features_cols if col in df_cluster_3.columns]

# El DataFrame principal solo contiene las columnas que varían en el tiempo.
ts_df = TimeSeriesDataFrame.from_data_frame(
    df_cluster_3[all_cols],
    id_column=item_id_col,
    timestamp_column=timestamp_col
)

# El DataFrame de características estáticas.
static_features_df = df_cluster_3.drop_duplicates(subset=[item_id_col])[[item_id_col] + static_feature_cols_present].set_index(item_id_col)
ts_df.static_features = static_features_df

print("\nVista previa del TimeSeriesDataFrame (listo para AutoGluon):")
print(ts_df.head())
print("\nCaracterísticas estáticas asociadas:")
print(ts_df.static_features)


Vista previa del TimeSeriesDataFrame (listo para AutoGluon):
                             tn  dias_laborables_arg  anio  mes  secuencia  \
item_id     timestamp                                                        
20003_10001 2017-01-01  100.213                   22  2017    1          1   
            2017-02-01  126.978                   18  2017    2          2   
            2017-03-01  114.529                   22  2017    3          3   
            2017-04-01   37.346                   18  2017    4          4   
            2017-05-01   76.664                   21  2017    5          5   

                        trimestre  
item_id     timestamp              
20003_10001 2017-01-01          1  
            2017-02-01          1  
            2017-03-01          1  
            2017-04-01          2  
            2017-05-01          2  

Características estáticas asociadas:
              cat1               cat1_cat2  \
item_id                                      
20003_100

In [ ]:
# ===================================================================
# 5. ENTRENAMIENTO DEL MODELO
# ===================================================================
# El último dato que tenemos es de 2019-12. Queremos predecir para 2020-02.
# Esto significa que necesitamos una predicción de 2 pasos hacia adelante (2020-01 y 2020-02).
prediction_length = 2

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-colab-known-covariates", # Carpeta para guardar los modelos
    target=target_col,
    eval_metric="MASE" # Una buena métrica robusta para series temporales
)

# Entrenamos el modelo. Le pasamos los datos de entrenamiento y las covariables CONOCIDAS
# correspondientes a ese período de entrenamiento. AutoGluon las usará para aprender la relación.
predictor.fit(
    ts_df,
    time_limit=360, # Límite de 6 minutos para el entrenamiento en Colab
    presets="medium_quality"
)

In [23]:
# ===================================================================
# 5. ENTRENAMIENTO DEL MODELO
# ===================================================================
# El último dato que tenemos es de 2019-12. Queremos predecir para 2020-02.
# Esto significa que necesitamos una predicción de 2 pasos hacia adelante (2020-01 y 2020-02).
prediction_length = 2

predictor = TimeSeriesPredictor(
    prediction_length=prediction_length,
    path="autogluon-colab-known-covariates", # Carpeta para guardar los modelos
    target=target_col,
    eval_metric="WAPE" # Una buena métrica robusta para series temporales
)
# WAPE = “¿Qué tan grandes son mis errores contra el total?”
# MASE = “¿Qué tanto mejoro respecto a un modelo trivial?”


In [25]:
# Entrenamos el modelo. Le pasamos los datos de entrenamiento y las covariables CONOCIDAS
# correspondientes a ese período de entrenamiento. AutoGluon las usará para aprender la relación.
predictor.fit(
    ts_df,
    time_limit=600, # Límite de 6 minutos para el entrenamiento en Colab
    presets="fast_training"
)
#Preset 'low_quality' was not found. Valid presets: ['best_quality', 'high_quality', 'medium_quality', 'fast_training', 'bolt_tiny', 'bolt_mini', 'bolt_small', 'bolt_base', 'chronos_tiny', 'chronos_mini', 'chronos_small', 'chronos_base', 'chronos_large', 'chronos_ensemble', 'chronos_large_ensemble', 'chronos', 'best', 'high', 'medium', 'bq', 'hq', 'mq']

Beginning AutoGluon training... Time limit = 600s
AutoGluon will save models to '/content/autogluon-colab-known-covariates'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          8
GPU Count:          1
Memory Avail:       48.56 GB / 50.99 GB (95.2%)
Disk Space Avail:   192.83 GB / 235.68 GB (81.8%)
Setting presets to: fast_training

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WAPE,
 'hyperparameters': 'very_light',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 600,
 'verbosity': 2}

Inferred time series frequency: 'MS'
Provided train_data 

## voy a generar simplemente el promedio de los últimos 12 meses por producto

In [ ]:
# a partir del df_cluster_4 genero df_resto con solamente la suma por date_mes de tn para cada product_id y product_id
df_resto = df_cluster_4.groupby(['product_id', 'date_mes'])['tn'].sum().reset_index()
df_resto.head()

,product_id,date_mes,tn
0,20001,2017-01,23.147
1,20001,2017-02,15.670
2,20001,2017-03,12.357
3,20001,2017-04,8.887
4,20001,2017-05,27.255


In [ ]:
df_resto.dtypes

,0
product_id,int64
date_mes,period[M]
tn,float64


In [ ]:
df_2018 = df_resto[(df_resto['date_mes'] >= '2018-01') & (df_resto['date_mes'] <= '2018-12')]

In [ ]:
# 3. Agrupamos por product_id y calculamos el promedio de tn en ese período
df_promedio = (
    df_2018.groupby('product_id', as_index=False)['tn']
    .mean()
    .rename(columns={'tn': 'tn_prom_2018'})
)

# 4. El resultado es un DataFrame con columnas: product_id | tn_prom_2018
print(df_promedio.head())

   product_id  tn_prom_2018
0       20001        23.111
1       20002        14.977
2       20003        13.621
3       20004        10.495
4       20005         5.497


In [ ]:
productos_validar.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [ ]:
productos_validar.shape

(780, 1)

In [ ]:
df_promedio.shape

(1051, 2)

In [ ]:
# Merge para obtener el promedio solo para los product_id que están en productos_validar
df_validar_promedio = productos_validar.merge(
    df_promedio,
    on='product_id',
    how='left'
)

# Ahora df_validar_promedio tendrá dos columnas: product_id y tn_prom_2018
print(df_validar_promedio.head())

   product_id  tn_prom_2018
0       20001        23.111
1       20002        14.977
2       20003        13.621
3       20004        10.495
4       20005         5.497


In [ ]:
df_validar_promedio.shape

(780, 2)

In [ ]:
df_validar_promedio.to_csv(DATOS_DIR +'df_cluster4.csv', index=False)